In [1]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.9.1 requires tensorflow~=2.16.1, but you have tensorflow 2.17.0 which is incompatible.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.17.0 which is incompatible.
tf-keras 2.16.0 

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import seaborn as sns

2024-10-20 13:27:30.273141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 13:27:30.293864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 13:27:30.300236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
data_dir = "/kaggle/input/plantvillage-subset/plantvillage dataset"

In [11]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3  
)


train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    shuffle=True
)


validation_gen = datagen.flow_from_directory(
    data_dir,  
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 11067 images belonging to 15 classes.
Found 4735 images belonging to 15 classes.


In [12]:
class ResetDataGeneratorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_gen.reset()
        validation_gen.reset()

In [13]:
import math

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x) 
x = Dense(1024, activation='relu')(x)  
predictions = Dense(train_gen.num_classes, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_gen,
    steps_per_epoch=math.ceil(train_gen.samples//train_gen.batch_size),
    validation_data=validation_gen,
    validation_steps=math.ceil(validation_gen.samples//validation_gen.batch_size),
    epochs=10,
    callbacks=[ResetDataGeneratorCallback()]
)

for epoch in range(len(history.history['accuracy'])):
    print(f"Epoch {epoch + 1}/{len(history.history['accuracy'])}")
    print(f"Training Accuracy: {history.history['accuracy'][epoch] * 100:.2f}%")
    print(f"Validation Accuracy: {history.history['val_accuracy'][epoch] * 100:.2f}%\n")


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


172/172 ━━━━━━━━━━━━━━━━━━━━ 242s 1s/step - accuracy: 0.6908 - loss: 1.0814 - val_accuracy: 0.9324 - val_loss: 0.2307
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9375 - loss: 0.2407 - val_accuracy: 1.0000 - val_loss: 0.0822
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 226s 1s/step - accuracy: 0.9436 - loss: 0.1974 - val_accuracy: 0.9518 - val_loss: 0.1547
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8906 - loss: 0.2029 - val_accuracy: 1.0000 - val_loss: 0.0369
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 226s 1s/step - accuracy: 0.9582 - loss: 0.1384 - val_accuracy: 0.9628 - val_loss: 0.1192
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9062 - loss: 0.2028 - val_accuracy: 1.0000 - val_loss: 0.0210
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 228s 1s/step - accuracy: 0.9676 - loss: 0.1087 - val_accuracy: 0.9638 - val_loss: 0.1066
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9688 - loss: 0.1420 - val_accuracy: 1.0000 -

In [14]:
from sklearn.metrics import classification_report

true_labels = validation_gen.classes

predictions = model.predict(validation_gen, steps=validation_gen.samples // validation_gen.batch_size + 1)

predicted_labels = np.argmax(predictions, axis=1)

class_labels = list(validation_gen.class_indices.keys())
print(classification_report(true_labels, predicted_labels, target_names=class_labels))


74/74 ━━━━━━━━━━━━━━━━━━━━ 74s 954ms/step
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.98      0.93      0.96       189
                                 Apple___Black_rot       0.99      0.97      0.98       186
                          Apple___Cedar_apple_rust       0.99      1.00      0.99        82
                                   Apple___healthy       0.96      1.00      0.98       493
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.89      0.81      0.85       153
                       Corn_(maize)___Common_rust_       1.00      1.00      1.00       357
               Corn_(maize)___Northern_Leaf_Blight       0.90      0.95      0.93       295
                            Corn_(maize)___healthy       1.00      1.00      1.00       348
                             Potato___Early_blight       0.99      0.99      0.99       300
                              Potato_

In [15]:
model.save('plant_disease_classifier.h5')